In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def preprocess_image(file_path, target_size=(224, 224)):
    img = load_img(file_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def get_class_names():
    return [
        "bibimbap", "bulgogi", "godeungeogui", "jjambbong", "ramyun",
        "yangnyumchicken", "duinjangjjigae", "gamjatang", "gimbap", "jeyukbokkeum",
        "jjajangmyeon", "kalguksu", "kimchijjigae", "mandu", "pajeon",
        "samgyetang", "samgyeopsal", "sundaegukbap", "tteokbokki", "tteokguk"
    ]

# Path to the saved model
model_path = 'C:/Users/dslab/OneDrive - GIST/바탕 화면/Desktop Backup/ReacApp/src/foods_saved_model.h5'

try:
    # Load the model
    model = tf.keras.models.load_model(model_path)
    print(f"Model loaded successfully from {model_path}")
    
    # Print model summary
    model.summary()

    # Test the model with a sample image
    test_image_path = 'C:/Users/dslab/OneDrive - GIST/바탕 화면/Desktop Backup/ReacApp/k1.jpg'  # Update this path to an actual image path
    img_array = preprocess_image(test_image_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    class_names = get_class_names()
    predicted_label = class_names[predicted_class]
    print(f'Predicted class: {predicted_label}')

except Exception as e:
    print(f"Error loading or testing the model: {e}")


Error loading or testing the model: 'utf-8' codec can't decode byte 0xd5 in position 36: invalid continuation byte


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define paths
train_images_dir = 'C:/Users/dslab/Downloads/train-20240522T054636Z-001/train/images'
labels_dir = 'C:/Users/dslab/Downloads/train-20240522T054636Z-001/train/labels'

# Function to create a dictionary of image paths and their corresponding labels
def create_labels_dict(labels_dir):
    labels_dict = {}
    for label_file in os.listdir(labels_dir):
        if label_file.endswith('.txt'):  # Assuming label files are .txt
            file_path = os.path.join(labels_dir, label_file)
            with open(file_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_id = int(parts[0])  # Assuming first part is the class_id
                    image_name = os.path.splitext(label_file)[0] + '.jpg'  # Assuming image extension is .jpg
                    labels_dict[image_name] = class_id
    return labels_dict

# Create labels dictionary
labels_dict = create_labels_dict(labels_dir)

# Get unique labels
unique_labels = list(set(labels_dict.values()))
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
index_to_label = {idx: label for label, idx in label_to_index.items()}

# Image data generator for preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Use 20% of data for validation
)

# Helper function to load images and labels
def load_data(image_dir, labels_dict, target_size=(224, 224)):
    images = []
    labels = []
    for image_name, class_id in labels_dict.items():
        image_path = os.path.join(image_dir, image_name)
        if os.path.exists(image_path):
            img = load_img(image_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(class_id)
    return np.array(images), np.array(labels)

# Load images and labels
images, labels = load_data(train_images_dir, labels_dict)

# Split into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(unique_labels))
y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(unique_labels))

# Define the model
# Load pre-trained MobileNetV2 model without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(unique_labels), activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Image data generator for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Image data generator for validation
val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
validation_generator = val_datagen.flow(x_val, y_val, batch_size=32)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Save the trained model
model.save('food_saved_model.h5')
# Assuming `model` is your trained model
model.save('foods_saved_model.h5')

# Making Predictions
from tensorflow.keras.preprocessing import image

# Define class names
class_names = [
    "bibimbap", "bulgogi", "godeungeogui", "jjambbong", "ramyun",
    "yangnyumchicken", "duinjangjjigae", "gamjatang", "gimbap", "jeyukbokkeum",
    "jjajangmyeon", "kalguksu", "kimchijjigae", "mandu", "pajeon",
    "samgyetang", "samgyeopsal", "sundaegukbap", "tteokbokki", "tteokguk"
]


Epoch 1/10
53/53 [==============================] - 25s 441ms/step - loss: 1.3586 - accuracy: 0.5842 - val_loss: 0.8007 - val_accuracy: 0.7678
Epoch 2/10
53/53 [==============================] - 24s 447ms/step - loss: 0.4883 - accuracy: 0.8428 - val_loss: 0.7265 - val_accuracy: 0.7891
Epoch 3/10
53/53 [==============================] - 24s 457ms/step - loss: 0.3130 - accuracy: 0.8956 - val_loss: 0.6590 - val_accuracy: 0.7891
Epoch 4/10
53/53 [==============================] - 24s 450ms/step - loss: 0.2206 - accuracy: 0.9318 - val_loss: 0.6670 - val_accuracy: 0.8081
Epoch 5/10
53/53 [==============================] - 24s 447ms/step - loss: 0.1471 - accuracy: 0.9531 - val_loss: 0.6502 - val_accuracy: 0.8057
Epoch 6/10
53/53 [==============================] - 24s 448ms/step - loss: 0.1191 - accuracy: 0.9614 - val_loss: 0.6439 - val_accuracy: 0.8270
Epoch 7/10
53/53 [==============================] - 24s 449ms/step - loss: 0.1013 - accuracy: 0.9692 - val_loss: 0.8352 - val_accuracy: 0.7701

c:\Users\dslab\anaconda3\envs\myenv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a simple model for demonstration purposes
model = Sequential([
    Dense(32, activation='relu', input_shape=(784,)),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Paths for the saved models
saved_model_dir = 'C:/Users/dslab/OneDrive - GIST/바탕 화면/Desktop Backup/ReacApp/src/saved_model.h5'
hdf5_model_path = 'model.h5'

# Remove the existing saved model directory if it exists
if os.path.exists(saved_model_dir):
    shutil.rmtree(saved_model_dir)

# Save the model in TensorFlow SavedModel format (as a directory)
model.save(saved_model_dir)

# Save the model in HDF5 format (as a single file)
model.save(hdf5_model_path)

# Loading the model in TensorFlow SavedModel format
model_saved = tf.keras.models.load_model(saved_model_dir)
print("Model loaded from SavedModel format")
model_saved.summary()

# Loading the model in HDF5 format
model_h5 = tf.keras.models.load_model(hdf5_model_path)
print("Model loaded from HDF5 format")
model_h5.summary()


NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/Users/dslab/OneDrive - GIST/바탕 화면/Desktop Backup/ReacApp/src/saved_model.h5'